This file contains the necessary code to get the counts and metrics for each company and then runs it through the various ML models

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LinearRegression
import numpy as np
import csv
import os
import pandas as pd
import math
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
import tflearn

In [ ]:
dir = 'C:/Users/Chaitu Konjeti/socweb-glassdoor-project/REVIEWS/'

In [ ]:
all_metrics = {}

metrics_file = pd.read_csv('metrics.csv')

# get all the proper metrics from the metrics file
for row in metrics_file.index:
    #print(int(metrics_file['Data Year - Fiscal'][row]))
    if metrics_file['Company Name'][row] not in all_metrics.keys():
        all_metrics[metrics_file['Company Name'][row]] = []
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])) and not int(metrics_file['Data Year - Fiscal'][row]) != '0':
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))
    else:
        if not math.isnan(metrics_file['Gross Profit (Loss)'][row]) and not math.isnan(int(metrics_file['Data Year - Fiscal'][row])):
            all_metrics[metrics_file['Company Name'][row]].append((int(metrics_file['Gross Profit (Loss)'][row]), str(int(metrics_file['Data Year - Fiscal'][row]))))

In [4]:
metrics = []
list_of_posts = []

#iterate through all files in the directory
for filename in os.listdir(dir):
    print(filename)
    df = pd.read_csv(dir + filename, header=0)

    #get dates and pros columns from data file
    dates = df['date']
    pros = df['pros']

    #get the list of years for all the posts in the file and create a dictionary with years as the keys
    years = [date.split(' ')[3] for date in dates]
    corpus = {key: '' for key in set(years)}

    #add the post to the proper year in the dictionary
    for date, pro in zip(years, pros):
        corpus[date] += pro + ' '

    #sort the keys in order
    keys = sorted(corpus)
    
    company_name = filename.split('.')[0]
    
    metric_years = [val[1] for val in set(all_metrics[company_name]) if val[1] in years]
    
    metrics.extend([val[0] for val in set(all_metrics[company_name]) if val[1] in years])
    
    #only use the metrics from years that present in the datafile
    for key in keys:
        if key in metric_years:
            list_of_posts.append(corpus[key])
    
#creates count vectorizer for posts
count_vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 3))
counts = count_vectorizer.fit_transform(list_of_posts)

#creates Tfidf vectorizer for posts
tfid_vectorizer = TfidfVectorizer(ngram_range=(1, 3), analyzer='word')
tfid = tfid_vectorizer.fit_transform(list_of_posts)

metrics = np.asarray(metrics)

AMERICAN AIRLINES GROUP INC.csv
APPLE INC.csv
AVNET INC.csv


In [5]:
print(tfid.shape)

(28, 8511)


In [6]:
print(metrics.shape)

(28,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(tfid, metrics, test_size=0.2, random_state=1)

In [8]:
clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)

C:\Users\Chaitu Konjeti\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
clf.score(X_train, y_train)

0.9545454545454546

In [10]:
print(clf.predict(X_test))

[ 3602 45640  3534  3534  3534  7628]


In [11]:
y_test

array([ 3362, 97586, 93515, 71867, 78483,  2982])

In [1]:
# X_train = X_train.toarray().flatten()
# X_test = X_test.toarray().flatten()

# tensorflow.reset_default_graph()

# input_layer = tflearn.input_data(shape=[None, 1000])
# net = tflearn.embedding(input_layer, input_dim=100000, output_dim=128)
# hid = tflearn.fully_connected(input_layer, 10, activation='tanh') # a hidden layer with 10 neurons
# output_layer = tflearn.fully_connected(hid, 2, activation='softmax')

# sgd = tflearn.SGD(learning_rate=0.04, lr_decay=0.96, decay_step=1000)
# net = tflearn.regression(output_layer, optimizer=sgd, loss='categorical_crossentropy')


# model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='tfdir')
# try:
#     model.fit(X_train, y_train, n_epoch=5, batch_size=100, show_metric=True, run_id="Imdb")
# except KeyboardInterrupt as e:
#     print("Stopped by user")